In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 14968429
paper_name = 'riles_reines_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/hits.txt', sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 44 x 6


In [7]:
original_data.head()

,ORF,Gene name wild-type,Process,Function,6AU sensitivity,MPA sensitivity
0,YGL043W,DST1,Transcription,Transcription elongation factor S-II,3,3
1,YMR070W,MOT3,Transcription,Transcriptional repressor and activator,3,1
2,YNR052C,POP2,Transcription,Transcription factor,3,3
3,YGL244W,RTF1,Transcription,Transcription elongation factor,3,3
4,YDR392W,SPT3,Transcription,SAGA histone acetyltransferase complex member,3,3


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene name wild-type, Process, Function, 6AU sensitivity, MPA sensitivity, orf]
Index: []


In [13]:
original_data['data'] = original_data['6AU sensitivity']

In [14]:
original_data.loc[original_data['data']=='Resistant','data'] = -1

In [15]:
original_data['data'] = -pd.to_numeric(original_data['data'], errors='coerce')

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(44, 1)

# Load & process tested strains

In [22]:
tested = pd.read_excel('raw_data/aHOMDIP_FinalList_2804_15Aug03.xlsx', sheet_name='box order', header=None)

In [23]:
tested.head()

,0
0,YAL002W
1,YAL004W
2,YAL005C
3,YAL007C
4,YAL008W


In [24]:
tested['orf'] = tested[0].astype(str)

In [25]:
tested['orf'] = clean_orf(tested['orf'])

In [28]:
tested.loc[tested['orf']=='YDL090','orf'] = 'YDL090C'

In [29]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [30]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [0, orf]
Index: []


In [31]:
tested_orfs = tested['orf'].unique()

In [32]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [33]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [34]:
data = original_data.copy()

In [35]:
dataset_ids = [188]
datasets = datasets.reindex(index=dataset_ids)

In [36]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [37]:
data.head()

dataset_id,188
data_type,value
orf,
YAL002W,0
YAL004W,0
YAL005C,0
YAL007C,0
YAL008W,0


## Subset to the genes currently in SGD

In [38]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [39]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,188
,data_type,value
gene_id,orf,
2,YAL002W,0
1863,YAL004W,0
4,YAL005C,0
5,YAL007C,0
6,YAL008W,0


# Normalize

In [40]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [41]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [42]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        188          
data_type       value    valuez
gene_id orf                    
2       YAL002W     0  0.030922
1863    YAL004W     0  0.030922
4       YAL005C     0  0.030922
5       YAL007C     0  0.030922
6       YAL008W     0  0.030922

# Print out

In [43]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [44]:
from IO.save_data_to_db3 import *

In [45]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 14968429...
Inserting the new data...


100%|██████████| 1/1 [00:04<00:00,  4.77s/it]

Updating the data_modified_on field...
